In [3]:
import http.client
import json
#from settings import API_KEY
import time
from pandas.io.json import json_normalize
import pandas as pd
import  csv
import ast
API_KEY = 'd91501d816e13b84b665bce29f571e7a'

## Funções

In [4]:
def get_url_credits(movie_id, api_key):
    return("/3/movie/" + str(movie_id) + "/credits" + "?api_key=" + str(api_key))

def get_json(conn, url):
	payload = "{}"
	conn.request("GET", url, payload)
	res = conn.getresponse()
	data = res.read()
	return(json.loads(data.decode("    utf-8")))

In [43]:
def get_movie_credits(ids):
    
	conn = http.client.HTTPSConnection("api.themoviedb.org")    
	movie_credits_df = pd.DataFrame()
    
	for movie_id in ids:
		url = get_url_credits(movie_id, API_KEY)
        
		try:
			credits_data = get_json(conn, url)
		except Exception as e:
			if (credits_data['status_code'] == 25):
				time.sleep(10)                
				credits_data = get_json(conn, url)        
        
		df = json_normalize(credits_data)
		movie_credits_df = movie_credits_df.append(df)

	return movie_credits_df

In [62]:
def get_cast(ids, movie_credits_df):

    movie_cast_df = pd.DataFrame()

    print(movie_credits_df['id'])
    for idx,movie_id in enumerate(ids,40):

        movie_item = pd.DataFrame()
        #print(movie_credits_df.head(90))
        print("movie_id: %s" % (movie_id))
        movie_item = movie_credits_df[movie_credits_df['id']==movie_id].reset_index()
        print(movie_item.head(90))
        print("%s of %s" % (idx,len(ids)))

        print(movie_item['cast'])
        
        for cast_id in movie_item['cast'][0]:
            cast_id.update({'movie_id': int(movie_id)})
            cast_id.pop('profile_path', None)
            movie_cast_df = movie_cast_df.append(cast_id, ignore_index=True)
    
    return movie_cast_df

## Testes

In [9]:
with open('out_sample.csv', 'r') as f:
#with open('out_all.csv', 'r') as f:    
    reader = csv.reader(f, skipinitialspace=True, delimiter=',')
    ids_list = list(reader)
    
ids = [item for sublist in ids_list for item in sublist]

In [10]:
len(ids)

156

In [11]:
# Transformando ids em numérico
ids_num = list(map(int, ids))

In [44]:
movie_credits_df = get_movie_credits(ids_num)

In [45]:
movie_credits_df.shape

(156, 5)

In [46]:
movie_credits_df = movie_credits_df.drop_duplicates('id', keep='first')

In [47]:
movie_credits_df['id'].isnull().sum()

1

In [48]:
movie_credits_df[movie_credits_df['id'].isnull()]

,cast,crew,id,status_code,status_message
0,NaN,NaN,NaN,25.0,Your request count (41) is over the allowed li...


In [49]:
movie_credits_df = movie_credits_df[movie_credits_df['status_code'] != 25]

In [50]:
movie_credits_df['id'].isnull().sum()

0

In [51]:
movie_credits_df = movie_credits_df.reset_index()

In [52]:
from IPython.display import display
pd.options.display.max_rows = None
movie_credits_df.head(90)

,index,cast,crew,id,status_code,status_message
0,0,"[{'name': 'Guy Pearce', 'id': 529, 'cast_id': ...","[{'name': 'Christopher Nolan', 'department': '...",77.0,NaN,NaN
1,0,"[{'name': 'Russell Crowe', 'id': 934, 'cast_id...","[{'name': 'Hans Zimmer', 'department': 'Sound'...",98.0,NaN,NaN
2,0,"[{'name': 'Christian Bale', 'id': 3894, 'cast_...","[{'name': 'Suzanne Smith', 'department': 'Prod...",1359.0,NaN,NaN
3,0,"[{'name': 'Tom Hanks', 'id': 31, 'cast_id': 3,...","[{'name': 'Alan Silvestri', 'department': 'Sou...",8358.0,NaN,NaN
4,0,"[{'name': 'Mel Gibson', 'id': 2461, 'cast_id':...","[{'name': 'Jake Eberts', 'department': 'Produc...",7443.0,NaN,NaN
5,0,"[{'name': 'Tom Cruise', 'id': 500, 'cast_id': ...","[{'name': 'Klaus Badelt', 'department': 'Crew'...",955.0,NaN,NaN
6,0,"[{'name': 'Benicio del Toro', 'id': 1121, 'cas...","[{'name': 'Guy Ritchie', 'department': 'Direct...",107.0,NaN,NaN
7,0,"[{'name': 'George Clooney', 'id': 1461, 'cast_...","[{'name': 'Wolfgang Petersen', 'department': '...",2133.0,NaN,NaN
8,0,"[{'name': 'Piper Perabo', 'id': 15555, 'cast_i...","[{'name': 'Jerry Bruckheimer', 'department': '...",6282.0,NaN,NaN
9,0,"[{'name': 'Matheus Nachtergaele', 'id': 8600, ...","[{'name': 'Guel Arraes', 'department': 'Direct...",40096.0,NaN,NaN


In [63]:
movie_cast_df = get_cast(ids_num, movie_credits_df)

0          77.0
1          98.0
2        1359.0
3        8358.0
4        7443.0
5         955.0
6         107.0
7        2133.0
8        6282.0
9       40096.0
10        146.0
11      11688.0
12       2024.0
13      10862.0
14      10393.0
15       9532.0
16         71.0
17       1907.0
18      16187.0
19       4247.0
20       4244.0
21       9679.0
22      10473.0
23       6521.0
24       4327.0
25      19457.0
26       2440.0
27       8859.0
28      10562.0
29       2123.0
30       3176.0
31       1493.0
32      10501.0
33      10867.0
34       8843.0
35      30970.0
36       9383.0
37        843.0
38      10873.0
39       9741.0
40        711.0
41      10647.0
42      13930.0
43      12212.0
44      12609.0
45      12211.0
46         55.0
47       1597.0
48       2085.0
49      10763.0
50      29076.0
51      10560.0
52       1900.0
53        134.0
54       3536.0
55      15653.0
56      11478.0
57      10567.0
58         16.0
59      10571.0
60      11826.0
61       3981.0
62      

IndexError: index out of bounds

In [63]:
ids_num[80]

10384

In [65]:
movie_credits_df.iloc[80]

cast              [{'id': 112, 'cast_id': 1, 'gender': 1, 'credi...
crew              [{'id': 7623, 'profile_path': '/uEAaCUJzyMI8N5...
id                                                             2046
status_code                                                     NaN
status_message                                                  NaN
Name: 0, dtype: object

In [64]:
movie_credits_df[movie_credits_df['id']==10384]

,cast,crew,id,status_code,status_message
0,"[{'id': 13548, 'cast_id': 6, 'gender': 2, 'cre...","[{'id': 1723, 'profile_path': '/gCU3dgnNaZuvmA...",10384.0,NaN,NaN


In [74]:
movie_credits_df.head(82)

,index,cast,crew,id,status_code,status_message
0,0,"[{'id': 529, 'movie_id': 77, 'cast_id': 4, 'ge...","[{'id': 525, 'profile_path': '/7OGmfDF4VHLLgbj...",77.0,NaN,NaN
1,0,"[{'id': 934, 'movie_id': 98, 'cast_id': 8, 'ge...","[{'id': 578, 'profile_path': '/oTAL0z0vsjipCru...",98.0,NaN,NaN
2,0,"[{'id': 3894, 'movie_id': 1359, 'cast_id': 3, ...","[{'id': 16378, 'profile_path': '/qlWi0kufzB7lC...",1359.0,NaN,NaN
3,0,"[{'id': 31, 'movie_id': 8358, 'cast_id': 3, 'g...","[{'id': 24, 'profile_path': '/isCuZ9PWIOyXzdf3...",8358.0,NaN,NaN
4,0,"[{'id': 2461, 'movie_id': 7443, 'cast_id': 3, ...","[{'id': 2359, 'profile_path': None, 'job': 'Ex...",7443.0,NaN,NaN
5,0,"[{'id': 500, 'movie_id': 955, 'cast_id': 31, '...","[{'id': 122, 'profile_path': '/j4BGxQq5klTT7hi...",955.0,NaN,NaN
6,0,"[{'id': 1121, 'movie_id': 107, 'cast_id': 19, ...","[{'id': 29, 'profile_path': None, 'job': 'Exec...",107.0,NaN,NaN
7,0,"[{'id': 1461, 'movie_id': 2133, 'cast_id': 16,...","[{'id': 5231, 'profile_path': '/t3Fxy4SBeOWj8T...",2133.0,NaN,NaN
8,0,"[{'id': 15555, 'movie_id': 6282, 'cast_id': 1,...","[{'id': 770, 'profile_path': '/p2rd1kgq7GiCx2n...",6282.0,NaN,NaN
9,0,"[{'id': 8600, 'movie_id': 40096, 'cast_id': 1,...","[{'id': 102560, 'profile_path': '/tAjR4ocM6Vjj...",40096.0,NaN,NaN
